# Exercise 4.3 Trajectories
prepared by M.Hauser

This is a short exercise showing how you can create lines colored according to a third value.

It is similar to an exercise prepared by Nicolas Piaget in the [c2sm python workshop](https://github.com/hvwaldow/pyws-BE-15-2-26).

It uses data calculated with the `lagranto` tool. `lagranto` is part of the Lagrangian Analysis Tool, Sprenger and Wernli, ([2015](https://www.geosci-model-dev.net/8/2569/2015/gmd-8-2569-2015.html)).

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import mplotutils as mpu

## Load data

Code from the Traj tool in the [Atmospheric Dynamics Group at ETH Zurich](http://www.iac.ethz.ch/groups/wernli).

In [ ]:
fN = "./../data/lsl_20070119_12_ana_48"


def read_trajectories(fN):
    with open(fN, "r") as fid:
        file_lines = fid.readlines()
    nvariables = file_lines[2].strip().split()
    head = file_lines[0].split()

    nhead = 5

    dtypes = ["f8" for var in nvariables]
    array = np.genfromtxt(
        fN, dtype=dtypes, names=nvariables, skip_header=nhead, missing_values=-999.99
    )

    timestep = float(array[1][0]) - float(array[0][0])
    period = float(array[-1][0]) - float(array[0][0])
    ntime = int(1 + np.around(period / timestep))
    ntra = int(array.size / ntime)

    # reshape traj file
    return np.reshape(array, (ntra, ntime))


traj = read_trajectories(fN)

`traj` is a [structured array](https://numpy.org/doc/stable/user/basics.rec.html), a special kind of numpy array (that I never use). It has named fields that can be accessed individually.

In [ ]:
print(f"{traj.shape=}")
print(f"{traj.dtype.names=}")

print("Field by name:")
traj["lon"]

### Exercise

 * plot the latitude and longitude of the trajectories
 
> each row in `traj` corresponds to one trajectory

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

lon = traj["lon"]
lat = traj["lat"]

# code here
n_tra = len(lon)
# for i in range(n_tra):
# plot here

### Solution

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

lon = traj["lon"]
lat = traj["lat"]

n_tra = len(lon)
for i in range(n_tra):
    ax.plot(lon[i], lat[i], color="0.5", transform=ccrs.PlateCarree())

By transposing them you can also directly plot all the lines:

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

lon = traj["lon"]
lat = traj["lat"]

ax.plot(lon.T, lat.T, color="0.5", transform=ccrs.PlateCarree());

## colorline

There is no direct way, to color lines depending on a 3rd value in pyplot. Therefore we start an internet search and find the following [notebook](http://nbviewer.jupyter.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb).

So we copy the code from there

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm


# Data manipulation:
def make_segments(x, y):
    """
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    """

    points = np.asarray([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    return segments


# Interface to LineCollection:


def colorline(
    x,
    y,
    z=None,
    ax=None,
    cmap="viridis",
    norm=None,
    linewidth=3,
    alpha=1.0,
    transform=None,
):
    """
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    """

    # parse default arguments

    # colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))

    if ax is None:
        ax = plt.gca()

    if isinstance(cmap, str):
        cmap = plt.get_cmap(cmap)

    if norm is None:
        norm = plt.Normalize(0.0, 1.0)

    if transform is None:
        transform = ax.transData

    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = [z]

    z = np.asarray(z)

    segments = make_segments(x, y)

    lc = LineCollection(
        segments,
        array=z,
        cmap=cmap,
        norm=norm,
        linewidth=linewidth,
        alpha=alpha,
        transform=transform,
    )

    ax.add_collection(lc)

    return lc

### Exercise

 * Try to get colorline to work
 * Bonus: use `mpu` to ensure the colorbar has the right height

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()
ax.set_extent([-90, 0, 20, 80], ccrs.PlateCarree())

# code here

# lc = colorline(...)

### Solution

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()
ax.set_extent([-90, 0, 20, 80], ccrs.PlateCarree())

lc = colorline(lon.T, lat.T, transform=ccrs.PlateCarree())


# create colorbar
cbar = mpu.colorbar(lc, ax, extend="both")

### Exercise

 * color the lines according to the potential temperature (`TH = traj['TH']`)
 * you will have to pass a `norm` to `colorline`
   * what do you have to choose for `norm=plt.Normalize(min, max)`?
 * use the colormap named `"Reds"`
 
Note
> you will have to loop through lat, lon and TH 

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()
ax.set_extent([-90, 0, 20, 80], ccrs.PlateCarree())

TH = traj["TH"]

# norm = plt.Normalize(...)

n_tra = len(lon)
for i in range(n_tra):
    # adjust colorline
    lc = colorline(lon[i], lat[i])


# create colorbar
cbar = mpu.colorbar(lc, ax, extend="both")

### Solution

Le't also add ticks and ticklabels. As this is a PlateCarree map, we can use the ordinary mechanism.

In [ ]:
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines(resolution="50m", color="0.4")

TH = traj["TH"]

norm = plt.Normalize(TH.min(), TH.max())

n_tra = len(lon)
for i in range(n_tra):
    lc = colorline(lon[i], lat[i], TH[i], norm=norm, linewidth=1, cmap="Reds")


ax.set_extent([-90, 0, 20, 80], ccrs.PlateCarree())

# create colorbar in this axes
cbar = mpu.colorbar(lc, ax, extend="both")
cbar.set_label("Potential Temperature (K)")


# set ticks
tick_lon = np.arange(-90, 1, 15)
tick_lat = np.arange(20, 81, 10)

# set the ticks
ax.set_xticks(tick_lon, crs=ccrs.PlateCarree())
ax.set_yticks(tick_lat, crs=ccrs.PlateCarree())

# format the ticks as e.g 60°W
ax.xaxis.set_major_formatter(LongitudeFormatter())
ax.yaxis.set_major_formatter(LatitudeFormatter())

# plt.savefig('ex4_3_trajectories.png', dpi=300)